In [ ]:

# Import Pytorch
import torch
from torch import nn

# Import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")